### DeepCTR Estimator with TFRecord

### Import Model

In [1]:
import tensorflow as tf 

from tensorflow.python.ops.parsing_ops import FixedLenFeature
from deepctr.estimator.inputs import input_fn_tfrecord
from deepctr.estimator.models import DeepFMEstimator

### Step 2 - Generate feature columns for linear part and dnn part

In [2]:
sparse_features = ['C' + str(i) for i in range(1, 27)]
dense_features = ['I' + str(i) for i in range(1, 14)]

In [3]:
dnn_feature_columns = []
linear_feature_columns = []

for i, feat in enumerate(sparse_features):
    dnn_feature_columns.append(tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_identity(feat, 1000), 4))
    
    linear_feature_columns.append(tf.feature_column.categorical_column_with_identity(feat, 1000))

for feat in dense_features:
    dnn_feature_columns.append(tf.feature_column.numeric_column(feat))
    linear_feature_columns.append(tf.feature_column.numeric_column(feat))

### Step 3: Generate the training samples with TFRecord format

In [16]:
feature_description = {k: FixedLenFeature(dtype=tf.int64, shape=1) for k in sparse_features}
feature_description.update(
    {k: FixedLenFeature(dtype=tf.float32, shape=1) for k in dense_features})
feature_description['label'] = FixedLenFeature(dtype=tf.float32, shape=1)

In [17]:
feature_description

{'C1': FixedLenFeature(shape=1, dtype=tf.int64, default_value=None),
 'C2': FixedLenFeature(shape=1, dtype=tf.int64, default_value=None),
 'C3': FixedLenFeature(shape=1, dtype=tf.int64, default_value=None),
 'C4': FixedLenFeature(shape=1, dtype=tf.int64, default_value=None),
 'C5': FixedLenFeature(shape=1, dtype=tf.int64, default_value=None),
 'C6': FixedLenFeature(shape=1, dtype=tf.int64, default_value=None),
 'C7': FixedLenFeature(shape=1, dtype=tf.int64, default_value=None),
 'C8': FixedLenFeature(shape=1, dtype=tf.int64, default_value=None),
 'C9': FixedLenFeature(shape=1, dtype=tf.int64, default_value=None),
 'C10': FixedLenFeature(shape=1, dtype=tf.int64, default_value=None),
 'C11': FixedLenFeature(shape=1, dtype=tf.int64, default_value=None),
 'C12': FixedLenFeature(shape=1, dtype=tf.int64, default_value=None),
 'C13': FixedLenFeature(shape=1, dtype=tf.int64, default_value=None),
 'C14': FixedLenFeature(shape=1, dtype=tf.int64, default_value=None),
 'C15': FixedLenFeature(shape

In [29]:
train_model_input = input_fn_tfrecord('./criteo_sample.tr.tfrecords', feature_description, 'label', batch_size=256,
                                      num_epochs=10, shuffle_factor=10)
test_model_input = input_fn_tfrecord('./criteo_sample.te.tfrecords', feature_description, 'label',
                                     batch_size=2 ** 14, num_epochs=1, shuffle_factor=0)


In [30]:
test_model_input

<function deepctr.estimator.inputs.input_fn_tfrecord.<locals>.input_fn()>

In [31]:
model = DeepFMEstimator(linear_feature_columns, dnn_feature_columns, task='binary')

model.train(train_model_input)
eval_result = model.evaluate(test_model_input)

print(eval_result)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpu5og33p1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:ten